In [31]:
from urllib.parse import urljoin

class QuizIterator():
    def __init__(self, address, pages):
        self._address = address
        self._index = 1
        self._pages = pages

    def __iter__(self):
        return self

    def __next__(self):
        if self._index <= self._pages:
            address = urljoin(self._address, str(self._index))
            self._index += 1
            return address
        else:
            raise StopIteration

In [32]:
PAGE = "https://quizack.com/django/questions-and-answers/"
for quiz in QuizIterator(PAGE, 2):
    print(quiz)

https://quizack.com/django/questions-and-answers/1
https://quizack.com/django/questions-and-answers/2


In [3]:
import requests

page_get_response = requests.get(url=PAGE)

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_get_response.text, 'html.parser')
soup2 = BeautifulSoup(page_get_response.text, 'html.parser')

In [73]:
from collections import namedtuple
Question = namedtuple("Question", ("number", "link", "text"))

def add_questions(soup_object):
    for block in soup_object.find_all(attrs={"class": "col-lg-6 col-md-6 col-sm-12 mt-2"}):
        question_html = block.contents[1].contents[1].contents[1].contents
        q_numb = question_html[1].text
        q_link = question_html[3].attrs["href"]
        q_text = question_html[3].contents[0].text
        
        q = Question(q_numb, q_link, q_text)
        print(q)
        
        break

In [74]:
questions = []

add_questions(soup)




Question(number='1.', link='https://quizack.com/django/mcq/which-is-fictional-http-request-method', text='Which is Fictional HTTP request method?')
